In [1]:
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=658).to(device)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=655).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
#Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)
Label_encoder.classes_ = np.load('./cl_classes1406.npy', allow_pickle=True)
#model.load_state_dict(torch.load("../../best_model3004/pytorch_model.bin", map_location=device))
model.load_state_dict(torch.load("../../best_model2705/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [2]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])

In [3]:
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')[0]

'1704'

In [4]:
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [21]:
print(predict_prob('трубы сантехнические пластиковые ', 5))

{'3917': 0.9508989453315735, '7304': 0.011040955781936646, '4009': 0.0049907136708498, '7411': 0.002056280616670847, '7306': 0.0019132409943267703}


In [8]:
print(predict_prob('трубы металлические ', 5))

{'7304': 0.9171484112739563, '7306': 0.014556351117789745, '7411': 0.009647378697991371, '3917': 0.004930668044835329, '8307': 0.0029827230609953403}


In [17]:
preds = predict_prob("гелевая ручка", 5)
print('Code', 'prob-ty', sep=':    ')
[print(i, f"{preds[i]:.{5}f}", sep=':    ') for i in preds]


Code:    prob-ty
9608:    0.14924
3923:    0.08421
8477:    0.07631
3926:    0.06465
4819:    0.04241


[None, None, None, None, None]

In [18]:
def consolelog(probs):
    print('Code', 'prob-ty', sep=':|    ')
    print('_________________')
    for each in probs:
        print(each, f"{probs[each]:.{5}f}", sep=':|    ')
        print('_________________')

In [25]:
consolelog(predict_prob("БЕЗАЛКОГОЛЬНЫЕ ГАЗИРОВАННЫЕ НАПИТКИ АРОМАТИЗИРОВАННЫЕ", 5))

Code:|    prob-ty
_________________
2106:|    0.89622
_________________
2202:|    0.01869
_________________
3402:|    0.00393
_________________
2009:|    0.00361
_________________
2105:|    0.00338
_________________


In [20]:
consolelog(predict_prob("КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО", 5))

Code:|    prob-ty
_________________
1704:|    0.99382
_________________
2106:|    0.00027
_________________
1905:|    0.00023
_________________
9602:|    0.00023
_________________
2918:|    0.00020
_________________


In [44]:
consolelog(predict_prob("ПЕРЧАТКИ ТРИКОТАЖНЫЕ МАШИННОГО ВЯЗАНИЯ, ПРОПИТАННЫЕ ИЛИ ПОКРЫТЫЕ ПЛАСТМАССОЙ", 5))

Code:|    prob-ty
_________________
6116:|    0.98108
_________________
4202:|    0.00181
_________________
6216:|    0.00057
_________________
4010:|    0.00040
_________________
6115:|    0.00032
_________________
